In [ ]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Utils
################################################################################
from masterDBGate import masterDBGate
from timeUtils import timestat
from fileIO import fileIO


################################################################################
## General
################################################################################
from pandas import Series, DataFrame

######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

# Install

In [ ]:
#!python setup.py install
mdbGate = masterDBGate()
disc = mdbGate.getDBDisc("SpotifyCharts")

# LastFM API

In [ ]:
from fileIO import fileIO
io = fileIO()
#data = io.get("/Users/tgadfort/discogs/artists-lastfmapi/data/Eminem.p")

In [ ]:
%autoreload
art = artistLastFMAPI()
retval = art.getData("/Volumes/Piggy/Discog/artists-lastfmapi/data/Eminem.p")

In [ ]:
%autoreload
from dbArtistsLastFMAPI import dbArtistsLastFMAPI
from dbArtistsParse import dbArtistsPickleHTML
dbAP = dbArtistsPickleHTML(dbArtistsLastFMAPI())
dbAP.parse(expr='< 0 Days', force=False)

In [ ]:
%autoreload
from artistLastFMAPI import artistLastFMAPI
art = artistLastFMAPI()
art.getData("/Users/tgadfort/discogs/artists-lastfmapi/76/81231361976.p").show()

In [ ]:

        
def getMediaCounts(media):
    amcc = artistDBMediaCountsClass()

    credittype = "Releases"
    if amcc.counts.get(credittype) == None:
        amcc.counts[credittype] = {}
    for creditsubtype in media.media.keys():
        amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))

    return amcc

savedir = "db/lastfm"
tsAll   = timestat("Creating DB Data")
Nmod    = 100

modValData = {}
N = len(artists)
for i,(artistID,artist) in enumerate(artists.items()):
    artistTracks = artist["Tracks"]
    artistAlbums = artist["Albums"]
    artistName   = artist["Name"]
    artistURL    = artist["URL"]
    generalData  = {"Type": artist["Type"]}

 
    mediaData = {}
    mediaName = "Tracks"
    mediaData[mediaName] = []
    for code, artistTrack in artistTracks.items():
        album        = artistTrack["Name"]
        albumURL     = artistTrack["URL"]
        albumArtists = [artistName]

        amdc = artistDBMediaDataClass(album=album, url=albumURL, artist=albumArtists, code=code, year=None)
        mediaData[mediaName].append(amdc)

 
    mediaData = {}
    mediaName = "Albums"
    mediaData[mediaName] = []
    for code, artistAlbum in artistAlbums.items():
        album        = artistAlbum["Name"]
        albumURL     = artistAlbum["URL"]
        albumArtists = [artistName]

        amdc = artistDBMediaDataClass(album=album, url=albumURL, artist=albumArtists, code=code, year=None)
        mediaData[mediaName].append(amdc)
        
        
    artist      = artistDBNameClass(name=artistName, err=None)
    meta        = artistDBMetaClass(title=None, url=artistURL)
    url         = artistDBURLClass(url=artistURL)
    ID          = artistDBIDClass(ID=artistID)
    pages       = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
    profile     = artistDBProfileClass(general=generalData)
    media       = artistDBMediaClass()
    media.media = mediaData
    mediaCounts = getMediaCounts(media)
    info        = artistDBFileInfoClass(info=None)

# Deezer API

In [ ]:
from dbArtistsBase import dbArtistsBase
from fileUtils import getBaseFilename
from fsUtils import isFile, setFile, setDir
from ioUtils import getFile, saveFile
from timeUtils import timestat
from sys import prefix
import urllib
from time import sleep
from webUtils import getHTML
from pandas import Series
    
from fileIO import fileIO
from fsUtils import fileUtil
            

In [ ]:
from dbArtistsParse import dbArtistsPickleAPI
from dbArtistsDeezerAPI import dbArtistsDeezerAPI
dbAP = dbArtistsPickleAPI(dbArtistsDeezerAPI())

In [ ]:
dbAP.parse(expr='< 0 Days', force=True)

In [ ]:
from dbArtistsBase import dbArtistsBase
from timeUtils import timestat
from pandas import Series    
from fileIO import fileIO
from fsUtils import fileUtil

In [ ]:
from dbArtistsDeezerAPI import dbArtistsDeezerAPI
dbAP = dbArtistsPickleAPI(dbArtistsDeezerAPI())
dbAP.parse(expr='< 0 Days', force=True)

In [ ]:
artistAlbums = dbAP.artistAlbums
artistTracks = dbAP.artistTracks
artistsData  = dbAP.artistsData

In [ ]:
        for artistID,artist in artistsData.items():
            artistAPIData = {"Name": artist.name, "URL": artist.link, "Type": artist.type, "ID": str(artist.id),
                             "Tracks": artistTracks.get(artist.id, {}), "Albums": artistAlbums.get(artist.id, {})}
            retval   = self.artist.getData(artistAPIData)


# AllMusic

In [ ]:
mdbGate = masterDBGate()
disc = mdbGate.getDBDisc("AllMusic")

In [ ]:
for modVal in range(100):
    modValData = Series(disc.getDBModValData(modVal))
    disc.saveDBModValData(idata=modValData, modVal=modVal)

In [ ]:
from collections import Counter

allMusicTabs = {}
for modVal in range(100):
    modValData = Series(disc.getDBModValData(modVal))
    for artistID,artistData in modValData.iteritems():
        try:
            tabs = artistData.profile.extra.get("Tabs")
        except:
            continue
        if isinstance(tabs, dict):
            for tabName,tabLink in tabs.items():
                if allMusicTabs.get(tabName) is None:
                    allMusicTabs[tabName] = {}
                allMusicTabs[tabName][artistID] = tabLink.href
                
    cnts = Counter({tabName: len(tabLinks) for tabName,tabLinks in allMusicTabs.items()})
    print("{0: <4}".format(modVal)," | ".join(["{0: <13}{1}".format(x[0],x[1]) for x in cnts.most_common()]))

In [ ]:
%autoreload
from artistAllMusic import artistAllMusic
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParseComposition import dbArtistsComposition
from dbArtistsParseSong import dbArtistsSong
from dbArtistsParseCredit import dbArtistsCredit

In [ ]:
dbAP = dbArtistsComposition(dbArtistsAllMusic())
dbAP.parse(0, expr='< 0 Days', force=True)

In [ ]:
dbAP = dbArtistsSong(dbArtistsAllMusic())
dbAP.parse(0, expr='< 0 Days', force=True)

In [ ]:
dbAP = dbArtistsCredit(dbArtistsAllMusic())
dbAP.parse(0, expr='< 0 Days', force=True)

In [ ]:
bsdata = getHTML("/Users/tgadfort/discogs/artists-allmusic/0/song/0000009100.p")

In [ ]:
s = Series({"x": "y"})
s

# AlbumOfTheYear Lists

In [ ]:
retval

In [ ]:
retval.getData()

# Genius Lists

In [ ]:
from webUtils import webUtil
from fileIO import fileIO
from glob import glob
web = webUtil()
io  = fileIO()


In [ ]:
from time import sleep
from masterDBGate import masterDBGate
dutils = masterDBGate().getDBUtil("Genius")

for ifile in glob("/Users/tgadfort/Desktop/Most Popular * Artists on Genius.html"):
    bsdata = io.get(ifile)
    genRefs = [ref.attrs['href'] for ref in bsdata.findAll("a") if ref.attrs.get('href') is not None]
    genRefs = [href for href in genRefs if "/artists/" in href]
    print(len(genRefs))
    for url in genRefs:
        artistID = dutils.getArtistID(url)
        savename = dutils.getArtistSavename(artistID)
        from fsUtils import fileUtil
        if fileUtil(savename).exists:
            continue
        print("="*150)
        print(artistID,'\t',url,'\t',savename)
        dutils.downloadArtistURL(url, savename, force=False)
        sleep(3)

In [ ]:
retval = web.download(genRefs[0])

In [ ]:

retval = web.download("https://genius.com/artists-index/a")

# Analyze Tabs For AllMusic

In [ ]:
from masterDBGate import masterDBGate
mdbGate = masterDBGate()
disc = mdbGate.getDBDisc("AllMusic")

In [ ]:
modValData['0000509000'].profile.extra["Tabs"]['Songs'].get()

In [ ]:
allMusicTabs = {}
for modVal in range(100):
    modValData = disc.getDBModValData(modVal)
    for artistID,artistData in modValData.iteritems():
        try:
            tabs = artistData.profile.extra.get("Tabs")
        except:
            continue
        if isinstance(tabs, dict):
            allMusicTabs[artistID] = {tabName: tabLink.href for tabName,tabLink in tabs.items()}
    print(modVal,len(allMusicTabs))

In [ ]:
from fileIO import fileIO
io = fileIO()
from pandas import Series
io.save(idata=Series(allMusicTabs), ifile="AllMusicTabs.p")

In [ ]:
dbmap["Artist"].getData(disc.getRawFilename('0000435400')).show()

# Ressurect DatPiff

In [ ]:
contentData = []
for listing in bsdata.findAll("div", {"class": "contentListing"}):
    contentItems = listing.findAll("div", {"class": "contentItem"})
    for i,contentItem in enumerate(contentItems):
        artistDiv  = contentItem.find("div", {"class": "artist"})
        artistName = artistDiv.text if artistDiv is not None else None
        titleDiv   = contentItem.find("div", {"class": "title"})
        titleRef   = titleDiv.find("a") if titleDiv is not None else None
        titleName  = titleRef.text if titleRef is not None else None
        titleURL   = titleRef.attrs['href'] if titleRef is not None else None
        ratings    = contentItem.findAll("div", {"class": "text"})
        
        itemValue = {"Title": titleName, "Artist": artistName, "Ref": titleURL}
        for rating in ratings:
            if "Rating:" in rating.text:
                img   = rating.find("img")
                value = img.attrs['alt'] if img is not None else None
                itemValue["Rating"] = value
            if "Listens:" in rating.text:
                span  = rating.find("span")
                value = span.text if span is not None else None
                itemValue["Listens"] = value

        if all([titleName,artistName,titleURL]):
            contentData.append(itemValue)
            



In [ ]:
from artistDatPiff import artistDatPiff
from dbUtils import utilsDatPiff
from dbBase import dbBase
import urllib
from urllib.parse import quote
from webUtils import getHTML, webUtil
from fsUtils import dirUtil, fileUtil
from fileIO import fileIO


##################################################################################################################
# Base Class
##################################################################################################################
class dbArtistsDatPiff:
    def __init__(self, debug=False):
        self.db     = "DatPiff"
        self.disc   = dbBase(self.db.lower())
        self.artist = artistDatPiff(self.disc)
        self.dutils = utilsDatPiff(self.disc)
        self.debug  = debug
        
        self.baseURL   = "https://www.datpiff.com/"        
        self.searchURL = "https://www.datpiff.com/mixtapes-search.php"
        self.io        = fileIO()
        self.web       = webUtil()
        
    
    def getSearchArtistURL(self, artist, page=1):
        url = urllib.parse.urljoin(self.searchURL, "?criteria={0}&sort=relevance&search=&search[]=title&search[]=artists&search[]=djs&p={1}".format(quote(artist),page))
        return url
    
        
    def searchForArtist(self, artist, force=False, debug=False):
        print("===================== Searching For {0} =====================".format(artist))
        url = self.getSearchArtistURL(artist,page=1)
        if url is None:
            raise ValueError("URL is None!")

        ## Download data
        print("  ==> {0}".format(url))
        retval = self.web.download(url)
        if retval.getCode() != 200:
            print("Error downloading {0}".format(url))

        datadir  = dirUtil(dbDP.disc.getArtistsDir()).join("data")
        savename = fileUtil(datadir).join("{0}-1.p".format(artist))
        self.io.save(idata=retval.getData(), ifile=savename)
        print("  ==> Saved Page-1 of [{0}] Search To [{1}]".format(artist,savename))
        
        
    def downloadRemainingPages(self, artist):
        datadir  = dirUtil(self.disc.getArtistsDir()).join("data")
        savename = fileUtil(datadir).join("{0}-1.p".format(artist))
        bsdata   = getHTML(savename)        
        pageData = bsdata.find("div", {"class": "pagination"})
        refs = pageData.findAll("a")
        refs = {ref.text: ref.attrs['href'] for ref in refs if ref.text.isdigit()}
        
        for pageNum,pageRef in refs.items():
            savename = fileUtil(datadir).join("{0}-{1}.p".format(artist,pageNum))
            if savename.exists():
                continue
            
            self.io.save(idata=retval.getData(), ifile=savename)
            print("  ==> Saved Page-1 of [{0}] Search To [{1}]".format(artist,savename))
            
        print(refs)

    

        

In [ ]:
from artistDatPiff import artistDatPiff
from pandas import DataFrame, Series
from dbUtils import utilsDatPiff
from dbBase import dbBase
import urllib
from urllib.parse import quote
from webUtils import getHTML, webUtil
from fsUtils import dirUtil, fileUtil
from fileIO import fileIO
from time import sleep


##################################################################################################################
# Base Class
##################################################################################################################
class dbArtistsDatPiff:
    def __init__(self, debug=False):
        self.db     = "DatPiff"
        self.disc   = dbBase(self.db.lower())
        self.artist = artistDatPiff(self.disc)
        self.dutils = utilsDatPiff(self.disc)
        self.debug  = debug
        
        self.baseURL   = "https://www.datpiff.com/"        
        self.searchURL = "https://www.datpiff.com/mixtapes-search.php"
        self.io        = fileIO()
        self.web       = webUtil()
        
    
    def getSearchArtistURL(self, artist, page=1):
        url = urllib.parse.urljoin(self.searchURL, "?criteria={0}&sort=relevance&search=&search[]=title&search[]=artists&search[]=djs&p={1}".format(quote(artist),page))
        return url
    
        
    def searchForArtist(self, artist, force=False, debug=False):
        print("===================== Searching For {0} =====================".format(artist))
        url = self.getSearchArtistURL(artist,page=1)
        if url is None:
            raise ValueError("URL is None!")

        print("  ==> {0}".format(url))
        retval = self.web.download(url)
        if retval.getCode() != 200:
            print("Error downloading {0}".format(url))
            return

        datadir  = dirUtil(dbDP.disc.getArtistsDir()).join("data")
        savename = fileUtil(datadir).join("{0}-1.p".format(artist))
        self.io.save(idata=retval.getData(), ifile=savename)
        print("  ==> Saved Page-1 of [{0}] Search To [{1}]".format(artist,savename))
        sleep(3)
        
        
    def downloadRemainingPages(self, artist):
        datadir  = dirUtil(self.disc.getArtistsDir()).join("data")
        savename = fileUtil(datadir).join("{0}-1.p".format(artist))
        bsdata   = getHTML(savename)        
        pageData = bsdata.find("div", {"class": "pagination"})
        refs = pageData.findAll("a")
        refs = {ref.text: ref.attrs['href'] for ref in refs if ref.text.isdigit()}
        print("Found {0} Pages For [{1}] Search".format(len(refs),artist))
        
        for pageNum,pageRef in refs.items():
            savename = fileUtil(datadir).join("{0}-{1}.p".format(artist,pageNum))
            if savename.exists():
                continue

            url = self.getSearchArtistURL(artist,page=pageNum)
            print("  ==> {0}".format(url))
            retval = self.web.download(url)
            if retval.getCode() != 200:
                print("Error downloading {0}".format(url))
                break

            self.io.save(idata=retval.getData(), ifile=savename)
            print("  ==> Saved Page-[{0}/{1}] of [{2}] Search To [{3}]".format(pageNum,len(refs),artist,savename))
            sleep(3)
            
            
    def createArtistData(self):
        contentData = []
        
        datadir  = dirUtil(self.disc.getArtistsDir()).join("data")
        files = glob("{0}/*.p".format(datadir))
        print("Found {0} Downloaded Search Pages".format(len(files)))
        for ifile in files:
            try:
                bsdata = getHTML(ifile)
            except:
                print("Error with {0}".format(ifile))
                continue
            for listing in bsdata.findAll("div", {"class": "contentListing"}):
                contentItems = listing.findAll("div", {"class": "contentItem"})
                for i,contentItem in enumerate(contentItems):
                    artistDiv  = contentItem.find("div", {"class": "artist"})
                    artistName = artistDiv.text if artistDiv is not None else None
                    titleDiv   = contentItem.find("div", {"class": "title"})
                    titleRef   = titleDiv.find("a") if titleDiv is not None else None
                    titleName  = titleRef.text if titleRef is not None else None
                    titleURL   = titleRef.attrs['href'] if titleRef is not None else None
                    ratings    = contentItem.findAll("div", {"class": "text"})

                    itemValue = {"Title": titleName, "Artist": artistName, "Ref": titleURL}
                    for rating in ratings:
                        if "Rating:" in rating.text:
                            img   = rating.find("img")
                            value = img.attrs['alt'] if img is not None else None
                            itemValue["Rating"] = value
                        if "Listens:" in rating.text:
                            span  = rating.find("span")
                            value = span.text if span is not None else None
                            itemValue["Listens"] = value

                    if all([titleName,artistName,titleURL]):
                        contentData.append(itemValue)

        self.contentDF = Series(contentData).apply(Series).drop_duplicates()

In [ ]:
dbDP = dbArtistsDatPiff()
dbDP.createArtistData()

In [ ]:
from fileIO import fileIO
io = fileIO()
io.save(idata=dbDP.contentDF, ifile="DatPiffData.p")

In [ ]:
dbDP = dbArtistsDatPiff()
artists = ["Jay", "2", "50", "Eminem", "Nas", "Clan", "Kanye", "Plan"]
artists = ["Boy", "Baby", "Presents", "Missy", "Timbaland", "Ja", "Da"]
artists = ["21", "Joe", "Star", "Frank", "Royal", "Red", "JP", "Barry"]
artists = ["Junior", "Mace", "Z", "Chief", "SD", "Trap"]
artists = ["South", "Coast", "East", "West", "North", "Jungle", "Common"]
artists = ["Drake", "Street", "City", "New"]
artists = ["Album", "Gang"]
artists = ["Carter", "Juice", "J", "Wiz"]
artists = ["G", "Gee", "Style", "New", "Love", "Live", "Big", "Pac"]
artists = ["Class", "Side", "Polo", "MC", "Kid"]
for artist in artists:
    dbDP.searchForArtist(artist)
    dbDP.downloadRemainingPages(artist)

In [ ]:
from glob import glob

datadir  = dirUtil(self.disc.getArtistsDir()).join("data")
files = glob("/Volumes/Piggy/Discog/artists-datpiff/data/*.p")
len(files)

In [ ]:
datadir  = dirUtil(dbDP.disc.getArtistsDir()).join("data")
files = glob("{0}/*.p".format(datadir))
len(files)

In [ ]:
vc.head(50)
vc = contentDF["Artist"].value_counts()

In [ ]:
dbDP.downloadRemainingPages("Future")

In [ ]:
savename

# AlbumOfTheYear

In [ ]:
from dbArtistID import artistIDBase
import re
    

##########################################################################
## Genius
##########################################################################
class artistIDGenius(artistIDBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        patterns  = [r'https://genius.com/artists/(\w+)']
        patterns += [r'/artists/*']
        patterns += [r'artists/*']
        self.patterns = patterns
        
    def getArtistID(self, s):
        self.s = str(s)
        
        ######################################################    
        ## Test For Format
        ######################################################
        self.testFormat(s)
        if self.err is not None:
            return None

        ######################################################    
        ## Pattern Matching
        ######################################################
        for pattern in self.patterns:
            print(pattern)
            groups = self.extractGroups(re.search(pattern, s))
            if groups is not None:
                print(groups)
                m = md5()
                for group in groups:
                    m.update(val.encode('utf-8'))
                hashval = m.hexdigest()
                artistID  = str(int(hashval, 16) % int(1e8))
                return artistID

In [ ]:
aid = artistIDGenius()
aid.getArtistID('https://genius.com/artists/Daichi-miura')

# Merge DBs

## Artist/Master Map

In [ ]:
from fileIO import fileIO
io = fileIO()

In [ ]:
ts = timestat("Getting Artist Discogs Data")
artistData = io.get("/Volumes/Seagate/DB/discogs_20211001_artists.p")
ts.stop()

In [ ]:
ts = timestat("Getting Artist Discogs Data")
albumsData = io.get("/Volumes/Seagate/DB/discogs_20211001_masters.p")
ts.stop()

In [ ]:
albumsData

In [ ]:
artistAlbumMap = {}
artistGenreStyleMap = {}
from collections import Counter
tsAll = timestat("Creating ArtistID -> Album/Genre/Style Mapping")

ts = timestat("Reindexing Albums Data For {0} Albums".format(albumsData.shape[0]))
for i,(albumID,albumData) in enumerate(albumsData.iterrows()):
    if (i+1) % 500000 == 0 or (i+1) == 100000 == 0:
        ts.update(n=i+1,N=albumsData.shape[0])
    albumArtistsData = albumData["MasterArtists"]
    albumValue       = {"MainReleaseID": albumData["main_release"], "Title": albumData["title"], "Year": albumData["year"]}
    genresData       = albumData["MasterGenres"]
    stylesData       = albumData["MasterStyles"]
    if isinstance(albumArtistsData,list):
        artistIDs = [artistData['id'] for artistData in albumArtistsData] if isinstance(masterData,list) else []
    
    for artistID in artistIDs:
        if artistAlbumMap.get(artistID) is None:
            artistAlbumMap[artistID] = {}
        artistAlbumMap[artistID][albumID] = albumValue

        if artistGenreStyleMap.get(artistID) is None:
            artistGenreStyleMap[artistID] = {"Genres": Counter(), "Styles": Counter()}
        for genre in genresData:
            artistGenreStyleMap[artistID]["Genres"][genre] += 1
        for style in stylesData:
            artistGenreStyleMap[artistID]["Styles"][style] += 1
            
ts.stop()

ts = timestat("Creating Genre/Style DataFrame")
artistGenreStyleMapDF = Series(artistGenreStyleMap).apply(Series)
artistGenreStyleMapDF["Genres"] = artistGenreStyleMapDF["Genres"].apply(lambda genres: [item[0] for item in genres.most_common(3)])
artistGenreStyleMapDF["Styles"] = artistGenreStyleMapDF["Styles"].apply(lambda styles: [item[0] for item in styles.most_common(3)])
io.save(idata=artistGenreStyleMapDF, ifile="/Volumes/Seagate/DB/discogs_20211001_ArtistGenreStyleMap.p")
ts.stop()

ts = timestat("Saving Artist/Album Data")
io.save(idata=Series(artistAlbumMap), ifile="/Volumes/Seagate/DB/discogs_20211001_ArtistAlbumMap.p")
ts.stop()

tsAll.stop()

## Artist/Master/Release Map

In [ ]:
from fileIO import fileIO
io = fileIO()

In [ ]:
artistAlbumMap = io.get("/Volumes/Seagate/DB/discogs_20211001_ArtistAlbumMap.p")

In [ ]:
artistAlbumMap['4120']

In [ ]:
from pandas import Series

ts = timestat("Getting Artist Discogs Data")
releaseData = io.get("/Volumes/Seagate/DB/discogs_20211001_releases-01.p")
ts.stop()

ts = timestat("Creating MasterID Columns")
mIDData = releaseData["master_id"].apply(lambda x: x[0] if isinstance(x,list) else None)
mIDData.name = "MasterID"
ts.update()
isMData = releaseData["master_id"].apply(lambda x: x[1][0][1] == 'true' if isinstance(x,list) else None)
isMData.name = "IsMaster"
ts.update()

releaseData = releaseData.join(mIDData).join(isMData).drop(['master_id'], axis=1)
ts.stop()

In [ ]:
releaseData[releaseData["MasterID"] == '2233']

In [ ]:
artistGenreStyleMapDF = Series(artistGenreStyleMap).apply(Series)

In [ ]:
artistGenreStyleMapDF["Genres"] = artistGenreStyleMapDF["Genres"].apply(lambda genres: [item[0] for item in genres.most_common(3)])
artistGenreStyleMapDF["Styles"] = artistGenreStyleMapDF["Styles"].apply(lambda styles: [item[0] for item in styles.most_common(3)])

In [ ]:
artistGenreStyleMapDF

In [ ]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass, artistDBFileInfoClass
from artistDBBase import artistDBTextClass, artistDBLinkClass
from strUtils import fixName
from dbUtils import utilsDiscogs


class artistDiscogsFull(artistDBBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        self.dutils = utilsDiscogs()
        
        
    ##############################################################################################################################
    ## Parse Data
    ##############################################################################################################################
    def getData(self, inputdata):
        for artistID,artistData in inputdata.iterrows():
            artistName  = inputdata['name']
            url         = "https://www.discogs.com/artist/{0}".format(artistID)
            
            generalData = {}
            generalData["RealName"]   = inputdata["realname"]
            generalData["Aliases"]    = inputdata["MasterAliases"]
            generalData["Groups"]     = inputdata["MasterGroups"]
            generalData["Members"]    = inputdata["MasterMembers"]
            generalData["Variations"] = inputdata["MasterNameVariations"]
            generalData = {k: v for k,v in generalData.items() if v is not None}
            generalData = generalData if len(generalData) > 0 else None
            
        artist      = artistDBNameClass(name=artistName, err=None)
        meta        = artistDBMetaClass(title=None, url=url)
        url         = artistDBURLClass(url=url)
        ID          = artistDBIDClass(ID=artistID)
        pages       = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        profile     = artistDBProfileClass(general=generalData)
        media       = self.getMedia()
        mediaCounts = self.getMediaCounts(media)
        info        = self.getInfo()
        
        adc = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, info=info)
        
        return adc
    
    
    ##############################################################################################################################
    ## File Info
    ##############################################################################################################################
    def getInfo(self):
        afi = artistDBFileInfoClass(info=self.fInfo)
        return afi
    

    ##############################################################################################################################
    ## Artist Name
    ##############################################################################################################################
    def getName(self):
        result1 = self.bsdata.find("h1", {'class':'hide_desktop'})
        result2 = self.bsdata.find("h1", {'class':'hide_mobile'})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            anc = artistDBNameClass(err=True)
            return anc

        if result:
            artist = result.text
            if len(artist) > 0:
                artist = fixName(artist)
                anc = artistDBNameClass(name=artist, err=None)
            else:
                result = self.bsdata.find("script", {"id": "artist_schema"})
                if result is None:
                    anc = artistDBNameClass(name=artist, err="Fix")
                else:
                    try:
                        artist = fixName(json.loads(result.text)["name"])
                        anc = artistDBNameClass(name=artist, err=None)
                    except:
                        anc = artistDBNameClass(name=artist, err="JSON")
        else:
            anc = artistDBNameClass(err="NoH1")

        return anc
    
    

    ##############################################################################################################################
    ## Meta Information
    ##############################################################################################################################
    def getMeta(self):
        metatitle = self.bsdata.find("meta", {"property": "og:title"})
        metaurl   = self.bsdata.find("meta", {"property": "og:url"})

        title = None
        if metatitle is not None:
            title = metatitle.attrs['content']

        url = None
        if metatitle is not None:
            url = metaurl.attrs['content']

        amc = artistDBMetaClass(title=title, url=url)
        return amc
        

    ##############################################################################################################################
    ## Artist URL
    ##############################################################################################################################
    def getURL(self):
        result1 = self.bsdata.find("link", {"rel": "canonical"})
        result2 = self.bsdata.find("link", {"hreflang": "en"})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            auc = artistDBURLClass(err=True)
            return auc

        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.discogs.com", "")
            if url.find("/artist/") == -1:
                url = None
                auc = artistDBURLClass(url=url, err="NoArtist")
            else:
                auc = artistDBURLClass(url=url)
        else:
            auc = artistDBURLClass(err="NoLink")

        return auc

    

    ##############################################################################################################################
    ## Artist ID
    ##############################################################################################################################
    def getID(self, suburl):
        discID   = self.dutils.getArtistID(suburl.url)
        aic = artistDBIDClass(ID=discID)
        return aic


    
    ##############################################################################################################################
    ## Artist Pages
    ##############################################################################################################################
    def getPages(self):
        from numpy import ceil
        bsdata = self.bsdata


        pageData = bsdata.find("div", {"class": "pagination bottom"})
        if pageData is None:
            err = "pagination bottom"
            apc = artistDBPageClass(err=err)
            return apc
        else:
            x = pageData.find("strong", {"class": "pagination_total"})
            if x is None:
                err = "pagination_total"
                apc = artistDBPageClass(err=err)
                return apc
            else:
                txt = x.text
                txt = txt.strip()
                txt = txt.replace("\n", "")
                retval = [tmp.strip() for tmp in txt.split('of')]

                try:
                    ppp   = int(retval[0].split('–')[-1])
                    tot   = int(retval[1].replace(",", ""))
                except:
                    err   = "int"
                    apc   = artistDBPageClass(err=err)
                    return apc

                if ppp < 500:
                    if tot < 25 or ppp == tot:
                        apc   = artistDBPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistDBPageClass(ppp=ppp, tot=tot, redo=True, more=False)
                else:
                    if tot < 500:
                        apc   = artistDBPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistDBPageClass(ppp=ppp, tot=tot, redo=False, more=True)
                        
                return apc
            
        return artistDBPageClass()
    
    

    ##############################################################################################################################
    ## Artist Variations
    ##############################################################################################################################
    def getProfile(self):  
        result = self.bsdata.find("div", {"class": "profile"})
        heads = result.findAll("div", {"class": "head"})
        content = result.findAll("div", {"class": "content"})
        profileData = dict(zip(heads, content)) if len(heads) == len(content) else {}
        generalData = {}
        for head,content in profileData.items():
            key  = head.text[:-1] if isinstance(head.text, str) else None
            refs = content.findAll("a")
            val  = [artistDBTextClass(content)] if len(refs) == 0 else [artistDBLinkClass(ref) for ref in refs]
            generalData[key] = val  


        extraData={}
        ulData = self.bsdata.findAll("ul", {"class": "facets_nav"})
        for ul in ulData:
            refs  = ul.findAll("a") if ul is not None else None
            attrs = [[ref.attrs.get('data-credit-type'), ref.attrs.get('data-credit-subtype'),ref] for ref in refs] if refs is not None else None
            for dctype,dcsubtype,ref in attrs:
                if not all([dctype,dcsubtype]):
                    continue
                if extraData.get(dctype) is None:
                    extraData[dctype] = {}
                extraData[dctype][dcsubtype] = artistDBLinkClass(ref)
        extraData = extraData if len(extraData) > 0 else None

        apc = artistDBProfileClass(general=generalData, extra=extraData)
        return apc

    
    
    ##############################################################################################################################
    ## Artist Media
    ##############################################################################################################################    
    def getMediaAlbum(self, td):
        amac = artistDBMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getMedia(self):
        amc = artistDBMediaClass()
        
        table = self.bsdata.find("table", {"id": "artist"})
        if table == None:
            amc.err="NoMedia"
            return amc

        name  = None
        for tr in table.findAll("tr"):
            h3 = tr.find("h3")
            if h3:
                name = h3.text
                amc.media[name] = []
                continue


            # Album, Class, Format
            result = tr.find("td", {"class": "title"})
            album  = None
            url    = None
            albumformat = name
            if result:
                retval      = self.getMediaAlbum(result)
                album       = fixName(retval.album)
                url         = retval.url
                albumformat = retval.aformat

            if album == None:
                continue

            # Code
            code = tr.attrs.get("data-object-id")

            # AlbumClass
            albumclass = tr.attrs.get("data-object-type")

            # AlbumURL
            result  = tr.find("td", {"class": "artist"})
            artists = None
            if result:
                artists = self.getNamesAndURLs(result)

            # Year
            result = tr.find("td", {"class": "year"})
            year   = None
            if result:
                year = result.text

            if name is None:
                name = "Albums"
                amc.media[name] = []
            amdc = artistDBMediaDataClass(album=album, url=url, aclass=albumclass, aformat=albumformat, artist=artists, code=code, year=year)
            amc.media[name].append(amdc)
            #if debug: print "  Found album:",album,"of type:",name


        if False:
            newMedia = {}
            for name,v in media.items():
                newMedia[name] = {}
                for item in v:
                    code = item['Code']
                    del item['Code']
                    newMedia[name][code] = item

            media = newMedia

        return amc
    
    

    ##############################################################################################################################
    ## Artist Media Counts
    ##############################################################################################################################
    def getMediaCounts(self, media):
        amcc = artistDBMediaCountsClass()
        
        results = self.bsdata.findAll("ul", {"class": "facets_nav"})
        if results is None or len(results) == 0:
            amcc.err = "No Counts"
            return amcc
            
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if amcc.counts.get(credittype) == None:
                                amcc.counts[credittype] = {}
                            if amcc.counts[credittype].get(creditsubtype) == None:
                                try:
                                    amcc.counts[credittype][creditsubtype] = int(count)
                                except:
                                    amcc.counts[credittype][creditsubtype] = count
                                    amcc.err = "Non Int"

        return amcc

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
# This downloads missing credit files
for modVal in range(100):
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=200)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

#for modVal in range(100):
#    dbAP = dbArtistsExtra(dbArtistsMusicBrainz())
#    dbAP.parse(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertExtra(dbArtistsMusicBrainz())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistExtras()

# Discogs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dometa = dbAP.parse(modVal, previousDays=50, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

if False:
    for modVal in range(100):
        break
        dbAP = dbArtistsExtra(dbArtistsDiscogs())
        dbAP.parse(modVal, previousDays=10, force=True)

    for modVal in range(100):
        break
        dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
        dbAP.parse(modVal, previousDays=10, force=True)

    for modVal in range(100):
        dbAP = dbArtistsPrimary(dbArtistsDiscogs())

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# AllMusic

In [ ]:
from ioUtils import getFile
dbLinks = getFile("dbLinks.p")
dbIDtoURL = getFile("dbIDToURLLookup.p")

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from pandas import DataFrame, Series
dbAM = dbArtistsAllMusic()
amRef = dbAM.disc.getMasterSlimArtistDiscogsDB()
print("I know about {0} IDs".format(amRef.shape))

amToGet = Series(list(dbLinks['allmusic'].keys()))
IDsToGet  = amToGet[~amToGet.isin(amRef.index)]
print("There are {0} IDs to get".format(len(IDsToGet)))

In [ ]:
from time import sleep
from fsUtils import isFile
from random import random
n = len(IDsToGet)
for i,artistID in enumerate(IDsToGet):
    url       = dbIDtoURL['allmusic'][artistID]
    savename  = dbAM.dutils.getArtistSavename(artistID)
    if not isFile(savename):
        print("{0: >3}/{1: <5}\t{2}\t{3}".format(i,n,savename,url))
        try:
            dbAM.dutils.downloadArtistURL(url=url, savename=savename, force=False)
        except:
            pass
        sleep(3*random())

# Genius Download List of URLs

In [ ]:
from ioUtils import getFile
dbLinks = getFile("dbLinks.p")
dbIDtoURL = getFile("dbIDToURLLookup.p")

In [ ]:
from dbArtistsGenius import dbArtistsGenius
from pandas import DataFrame, Series
dbGen = dbArtistsGenius()
genRef = dbGen.disc.getMasterSlimArtistDiscogsDB()
print("I know about {0} IDs".format(genRef.shape))

genToGet = Series(list(dbLinks['genius'].keys()))
IDsToGet  = genToGet[~genToGet.isin(genRef.index)]
print("There are {0} IDs to get".format(len(IDsToGet)))

In [ ]:
from time import sleep
from fsUtils import isFile
from random import random
n = len(IDsToGet)
for i,artistID in enumerate(IDsToGet):
    url       = dbIDtoURL['genius'][artistID]
    savename  = dbGen.dutils.getArtistSavename(artistID)
    if not isFile(savename):
        print("{0: >3}/{1: <5}\t{2}\t{3}".format(i,n,savename,url))
        try:
            dbGen.dutils.downloadArtistURL(url=url, savename=savename, force=False)
        except:
            pass
        sleep(2*random())

In [ ]:
for url in urls:
    print(url)
    dp = dbArtistsGenius()
    dp.downloadArtistFromURL(url)
    sleep(2)


In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Deezer

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# LastFM

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsLastFM import dbArtistsLastFM
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
lfm = dbArtistsLastFM()

In [ ]:
for modVal in range(100):
    try:
        lfm.assertDBModValExtraData(modVal, maxPages=4, test=False)
    except:
        pass

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsLastFM())
dometa = dbAP.parse(1, previousDays=1, force=True)

In [ ]:
dbAP = dbArtistsExtra(dbArtistsLastFM())
dbAP.parse(1, previousDays=1)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsExtra(dbArtistsLastFM())
    dbAP.parse(modVal, previousDays=3)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dometa = dbAP.parse(modVal, previousDays=3, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# AlbumOfTheYear

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAlbumOfTheYear import dbArtistsAlbumOfTheYear
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAlbumOfTheYear())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Genius

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsGenius())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# KWorbSpotify

In [ ]:
%autoreload
from dbArtistsKWorbSpotify import dbArtistsKWorbSpotify
from dbUtils import utilsKWorbSpotify
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML, dbArtistsRawFiles

In [ ]:
dbAP = dbArtistsKWorbSpotify()
dbAP.downloadKWorbSpotifyArtists(update=True)

In [ ]:
%autoreload
from webUtils import getHTML
from searchUtils import findExt
dbAP = dbArtistsRawFiles(dbArtistsKWorbSpotify(), "data")
dbAP.parse(previousDays=10)

# KWorbiTunes

In [ ]:
%autoreload
from dbArtistsKWorbiTunes import dbArtistsKWorbiTunes
from dbUtils import utilsKWorbYouTube
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML, dbArtistsRawFiles

In [ ]:
dbAP = dbArtistsKWorbiTunes()
dbAP.downloadKWorbiTunesArtists(update=True)

In [ ]:
%autoreload
from webUtils import getHTML
from searchUtils import findExt
dbAP = dbArtistsRawFiles(dbArtistsKWorbiTunes(), "data")
dbAP.parse(previousDays=10)

In [ ]:
%autoreload
from webUtils import getHTML
from searchUtils import findExt
dbAP = dbArtistsRawFiles(dbArtistsKWorbYouTube(), "data")
dbAP.parse(previousDays=10)

In [ ]:
%autoreload
from dbArtistsKWorbYouTube import dbArtistsKWorbYouTube
from dbUtils import utilsKWorbYouTube
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML, dbArtistsRawFiles

In [ ]:
bsdata = getHTML("/Volumes/Piggy/Discog/artists-kworbyoutube/data/justinbieber.p")
bsdata

In [ ]:
retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-kworbitunes/data/theweeknd.p")
retval.show()

In [ ]:
"/Volumes/Piggy/Discog/artists-kworbyoutube/data/beyoncé.p".encode('utf-8')

In [ ]:
bsdata = getHTML("/Volumes/Piggy/Discog/artists-kworbitunes/data/theweeknd.p")
bsdata

# RockCorner

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRockCorner())
    dbAP.parse(modVal, force=True)

# Rate Your Music

In [ ]:
"https://www.last.fm/music/Michael%2BBubl%25C3%25A9"

In [ ]:
url="Michael%Bublé"
print(quote(url, encoding='utf-8'))
print(utils.getArtistID(url, debug=True))

In [ ]:
https://www.last.fm/music/Michael%25%2BBubl%25C3%25A9/+albums?order=release_date

In [ ]:
quote(url)
utils.getArtistID("https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date", debug=True)

In [ ]:
#help(quote)

In [ ]:

#url="https://www.last.fm/music/Michael+Bublé"
url="https://www.last.fm/music/Dave+Matthews+Band"
url="https://www.last.fm/music/16+Deluxe"                               # 53342847883
from urllib.parse import quote
print(url)
print(quote(url))
#url="https://www.last.fm/music/16+Deluxe/+albums?order=release_date"   # 53342847883
url="https://www.last.fm/music/16%2BDeluxe/+albums?order=release_date" # 99404621521
#url="https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date"
#url="https://www.last.fm/music/Michael%25252BBubl%25C3%25A9/+albums?order=release_date"
#url="https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date"
from unicodedata import normalize
#‘NFC’, ‘NFKC’, ‘NFD’, and ‘NFKD’.
print(url)
print(normalize("NFC", url))
print(normalize("NFKC", url))
print(normalize("NFD", url))
print(normalize("NFKD", url))
print(utils.getArtistID(url))
print(utils.getArtistID(normalize("NFC", url)))
print(utils.getArtistID(normalize("NFKC", url)))
print(utils.getArtistID(normalize("NFD", url)))
print(utils.getArtistID(normalize("NFKD", url)))

#ifile="/Volumes/Piggy/Discog/artists-lastfm/47/92143197647.p"
#from artistLastFM import artistLastFM
#alm = artistLastFM()
#retval = alm.getData(ifile)
#retval.show()

In [ ]:
#data = getHTML("/Volumes/Piggy/Discog/artists-lastfm/47/85771689147.p")
#data

In [ ]:

ifile="/Volumes/Piggy/Discog/artists-lastfm/50/86362214250.p"
from artistLastFM import artistLastFM
alm = artistLastFM()
1/0
try:
    retval = alm.getData(ifile)
    retval.show()
except:
    print("HI")

In [ ]:



from dbUtils import utilsLastFM
utils = utilsLastFM()
print(utils.getArtistID(url))
ifile = "/Volumes/Piggy/Discog/artists-lastfm/89/83689536289.p"

url = "https://www.last.fm/music/Michael+Bublé"
print(utils.getArtistID(url))

In [ ]:
ifile="/Volumes/Piggy/Discog/artists-lastfm/0/96441297300.p"

In [ ]:
from ioUtils import saveFile, getsFile
from artistLastFM import artistLastFM
alm = artistLastFM()
retval = alm.getData(ifile)
retval.show()

In [ ]:
from dbArtistsDiscogs import dbArtistsDiscogs
for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=1000)

In [ ]:
dbAP.parse(modVal, previousDays=1000)

# RockCorner

In [ ]:
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsRockCorner())
dbAP.parse(69, previousDays=50)

In [ ]:
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsRockCorner())

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
dbAP = dbArtistsCredit(dbArtistsAllMusic())
dbAP.parse(69, previousDays=50)

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
dbAP = dbArtistsExtra(dbArtistsAllMusic())
dbAP.getArtistFiles(69, force=True)

In [ ]:

        saveIt = 0
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info

               
        forceSave = False
        if saveIt > 0 and doExtra is True:
            print("\tCalling Extra Parsing")
            dbdata = self.parseArtistModValExtraFiles(modVal, dbdata=dbdata, force=force, debug=debug)
            forceSave = True
            saveIt = len(dbdata)
            
         
        if len(self.creditToDownload) > 0:
            print("Found {0} artists that need new downloads.".format(len(self.creditToDownload)))
        

        if saveIt > 0 or forceSave is True:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("Saving {0} total artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    
        
        
        
                for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

    

In [ ]:

        
        
                    

        
def dbArtistsModValBase:
    def __init__(self, debug=False):
        self.debug = debug
        
    def findFiles(self):

        
    def 
            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info


In [ ]:
url="https://kworb.net/itunes/extended.html"
savename="kworb.p"
data, code = downloadURL(url)
saveFile(idata=data, ifile=savename)

In [ ]:
bsdata = getHTML(savename)

In [ ]:
bsdata

In [ ]:
url="https://kworb.net/itunes/artist/theweeknd.html"
savename="kworb_weeknd.p"
data, code = downloadURL(url)
saveFile(idata=data, ifile=savename)

In [ ]:
bsdata = getHTML(savename)

In [ ]:
ak = artistKWorb()
retval = ak.getData("kworb_weeknd.p")
retval.show()

In [ ]:
dp = dbArtistsKWorb()

In [ ]:
bsdata = getHTML("kworb_spotify.p")

In [ ]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBURLInfo, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass
from strUtils import fixName
from dbUtils import utilsKWorb
from webUtils import removeTag


class artistKWorb(artistDBBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        self.dbUtils = utilsKWorb()
        
        
    ##############################################################################################################################
    ## Parse Data
    ##############################################################################################################################
    def getData(self, inputdata):
        self.getDataBase(inputdata)
        self.checkData()
        
        artist      = self.getName()
        meta        = self.getMeta()
        url         = self.getURL()
        ID          = self.getID(artist)
        pages       = self.getPages()
        profile     = self.getProfile()
        media       = self.getMedia(artist, url)
        mediaCounts = self.getMediaCounts(media)
        
        err = [artist.err, meta.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc
    
    

    ##############################################################################################################################
    ## Artist Name
    ##############################################################################################################################
    def getName(self):
        #<div class="subcontainer"><span class="pagetitle">The Weeknd | <a href="/itunes/">Back to index</a> | <a href="/youtube/artist/theweeknd.html">YouTube stats</a> | 02/06 19:35 EST</span></div>
        span = self.bsdata.find("span", {"class": "pagetitle"})
        if artistData is None:
            anc = artistDBNameClass(err="NoSpan")
            return anc
        
        text = span.text.split(" | ")
        artistName = text[0]
        anc = artistDBNameClass(name=artistName, err=None)
        return anc
    
    

    ##############################################################################################################################
    ## Meta Information
    ##############################################################################################################################
    def getMeta(self):
        metatitle = self.bsdata.find("meta", {"property": "og:title"})
        metaurl   = self.bsdata.find("meta", {"property": "og:url"})

        title = None
        if metatitle is not None:
            title = metatitle.attrs['content']

        url = None
        if metatitle is not None:
            url = metaurl.attrs['content']

        amc = artistDBMetaClass(title=title, url=url)
        return amc
        

    ##############################################################################################################################
    ## Artist URL
    ##############################################################################################################################
    def getURL(self):
        #<div class="subcontainer"><span class="pagetitle">The Weeknd | <a href="/itunes/">Back to index</a> | <a href="/youtube/artist/theweeknd.html">YouTube stats</a> | 02/06 19:35 EST</span></div>
        span = self.bsdata.find("span", {"class": "pagetitle"})
        if artistData is None:
            anc = artistDBNameClass(err="NoSpan")
            return anc
        
        ref
        text = span.text.split(" | ")
        artistName = text[0]
        anc = artistDBNameClass(name=artistName, err=None)
        return anc
        
        artistData = self.bsdata.find("meta", {"property": "og:url"})
        if artistData is None:
            auc = artistDBURLClass(err=True)
            return auc
        
        url = artistData.attrs["content"]
        if url.find("/artist/") == -1:
            url = None
            auc = artistDBURLClass(url=url, err="NoArtist")
        else:
            auc = artistDBURLClass(url=url)

        return auc

    

    ##############################################################################################################################
    ## Artist ID
    ##############################################################################################################################
    def getID(self, artist):
        discID = self.dbUtils.getArtistID(artist.name)
        discID = None
        aic = artistDBIDClass(ID=discID)
        return aic


    
    ##############################################################################################################################
    ## Artist Pages
    ##############################################################################################################################
    def getPages(self):
        apc   = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        return apc
    
    

    ##############################################################################################################################
    ## Artist Variations
    ##############################################################################################################################
    def getProfile(self):
        data    = {}
        apc = artistDBProfileClass(profile=data.get("Formed"), aliases=data.get("Aliases"),
                                 members=data.get("Members"), groups=data.get("In Groups"),
                                 sites=data.get("Sites"), variations=data.get("Variations"))
        return apc

    
    
    ##############################################################################################################################
    ## Artist Media
    ##############################################################################################################################
    def getMediaAlbum(self, td):
        amac = artistDBMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getMedia(self, artist):
        amc  = artistDBMediaClass()
        
        mediaType = "Albums"
        amc.media[mediaType] = []

        for table in self.bsdata.findAll("table"):
            trs = table.findAll("tr")
            for itr,tr in enumerate(trs):
                tds = tr.findAll("td")
                for itd,td in enumerate(tds):
                    div=td.find("div", {"class": "wrap"})
                    if div is not None:
                        name = div.text
                        if name.startswith("Album: "):
                            mediaType = "Album"
                            title = name[7:]
                        else:
                            mediaType = "Single"
                            title = name
                        if media.get(mediaType) is None:
                            media[mediaType] = []

                        amdc = artistDBMediaDataClass(album=title, url=None, aclass=None, aformat=None, artist=artist, code=None, year=None)
                        if amc.media.get(mediaType) is None:
                            amc.media[mediaType] = []
                        amc.media[mediaType].append(amdc)                            

        return amc
    
    

    ##############################################################################################################################
    ## Artist Media Counts
    ##############################################################################################################################
    def getMediaCounts(self, media):
        
        amcc = artistDBMediaCountsClass()
        
        credittype = "Releases"
        if amcc.counts.get(credittype) == None:
            amcc.counts[credittype] = {}
        for creditsubtype in media.media.keys():
            amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))
            
        return amcc

In [ ]:
#from artistKWorb import artistKWorb
from dbUtils import utilsKWorb
from dbBase import dbBase
import urllib
from urllib.parse import quote
from webUtils import getHTML
from fsUtils import isFile


##################################################################################################################
# Base Class
##################################################################################################################
class dbArtistsKWorb:
    def __init__(self, debug=False):
        self.db     = "KWorb"
        self.disc   = dbBase(self.db.lower())
        self.artist = artistKWorb(self.disc)
        self.dutils = utilsKWorb(self.disc)
        self.debug  = debug
        
        self.baseURL   = "https://www.kworb.com/"        
        self.searchURL = "https://www.kworb.com/search/"

        
        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURLCredit(self, artistRef, page=1):
        url = "{0}/credits".format(artistRef)
        return url
    
    def getArtistURL(self, artistRef, page=1):
        url = "{0}/discography/all".format(artistRef)
        return url
        
        if artistRef.startswith("http"):
            return artistRef
        else:
            baseURL = self.baseURL
            url     = urllib.parse.urljoin(baseURL, quote(artistRef))
            return url

        if artistRef.startswith("/artist/") is False:
            print("Artist Ref needs to start with /artist/")
            return None
        
        baseURL = self.baseURL
        url     = urllib.parse.urljoin(baseURL, quote(artistRef))
        url     = urllib.parse.urljoin(url, "?sort=year%2Casc&limit=500") ## Make sure we get 500 entries)
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url

        
    
    ##################################################################################################################
    #
    # Search Functions
    #
    ##################################################################################################################
    def parseSearchArtist(self, artist, data, maxArtists=99, force=False, debug=False):
        if data is None:
            return None
        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "search-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist"})
            for li in lis:
                divs = li.findAll("div", {"class": "name"})
                for div in divs:
                    link     = div.find("a")
                    href     = link.attrs['href']
                    tooltip  = link.attrs['data-tooltip']
                    try:
                        from json import loads
                        tooltip = loads(tooltip)
                        artistID = tooltip['id']
                    except:
                        artistID = None
            
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": artist}
                    artistDB[href]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
            if iArtist > maxArtists:
                break
        
            discID   = self.dutils.getArtistID(href)
            url      = self.getArtistURL(href)
            savename = self.dutils.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                if force is False:
                    continue

            self.dutils.downloadArtistURL(url, savename, force=force)
            
    
    def getSearchArtistURL(self, artist):
        baseURL = self.searchURL
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("artists/", quote(artist)))                  
        return url
    
        
    def searchForArtist(self, artist, maxArtists=99, force=False, debug=False):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        url = self.getSearchArtistURL(artist)
        if url is None:
            raise ValueError("URL is None!")
                    
        ## Download data
        data, response = self.dutils.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        self.parseSearchArtist(artist, data, maxArtists, force, debug)
    
        
        
    ##################################################################################################################
    #
    # Search Functions (w/ Credit)
    #
    ##################################################################################################################
    def searchForArtistCredit(self, artist, artistID, force=False):
        print("\n\n===================== Searching For {0} , {1} =====================".format(artist, artistID))
        url = self.getSearchArtistURL(artist)
        if url is None:
            raise ValueError("URL is None!")
                    
        ## Download data
        data, response = self.dutils.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False
    
        self.parseSearchArtistCredit(artist, artistID, data, force)
        
      
    def parseSearchArtistCredit(self, artist, artistID, data, force=False):
        if data is None:
            return None
        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "search-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist"})
            for li in lis:
                divs = li.findAll("div", {"class": "name"})
                for div in divs:
                    link     = div.find("a")
                    href     = link.attrs['href']
                    tooltip  = link.attrs['data-tooltip']
                    try:
                        from json import loads
                        tooltip = loads(tooltip)
                        linkartistID = tooltip['id']
                    except:
                        linkartistID = None
            
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": artist}
                    artistDB[href]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            discID   = self.dutils.getArtistID(href)
            if discID != artistID:
                continue
            url      = self.getArtistURLCredit(href)
            savename = self.dutils.getArtistSavename(discID, credit=True)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                if force is False:
                    continue

            self.dutils.downloadArtistURL(url, savename, force=force)

In [ ]:
for table in bsdata.findAll("table"):
    for tr in table.findAll("tr"):
        ref = tr.find("a")
        if ref:
            url  = ref.attrs['href']
            name = ref.text
        print(tr)
        for td in tr.findAll("td"):
            print(td)
            1/0

In [ ]:
dbArtists = dbArtistsAllMusic()
        dbdata[dbArtists.db] = [dbArtists.disc, dbArtists, dbArtists.artist, dbArtists.dutils]


In [ ]:
from fsUtils import setDir, isDir, mkDir, mkSubDir, setFile, isFile, setSubFile
from ioUtils import saveFile, getFile
from fileUtils import getBaseFilename
from searchUtils import findExt
import urllib
from datetime import datetime, timedelta




class dbArtistsBase():
    def __init__(self, db, disc, artist, dutils, basedir=None, debug=False):
            
        #######################
        ## General Imports
        #######################
        self.db     = db
        self.disc   = disc
        self.name   = "artists"
        self.artist = artist
        self.dutils = dutils
        self.sleeptime=2
        self.debug  = debug
        
        self.getArtistsDir       = self.disc.getArtistsDir
        self.getArtistsDBDir     = self.disc.getArtistsDBDir
        self.getDiscogDBDir      = self.disc.getDiscogDBDir
        
        self.prevSearches        = {}
        
        self.modVal = self.disc.getMaxModVal
        
        self.creditToDownload = {}
        
        self.artistIDtoRefData = None
            
    
    ###############################################################################
    # Artist Data
    ###############################################################################
    def getData(self, ifile):
        info = self.artist.getData(ifile)
        return info
    
    def getFileData(self, artistID):
        ifile = self.getArtistSavename(artistID, 1)
        info  = self.getData(ifile)
        return info
    
    
    ###############################################################################
    # ModVals
    ###############################################################################
    def getModVals(self):
        return self.disc.getModValList()
        #return [str(x) for x in range(self.disc.getMaxModVal)]
    
    def getModValDirs(self):
        modVals = self.getModVals()
        retval  = [setDir(self.getArtistsDir(), str(modVal)) for modVal in modVals]
        return retval
    
    
    ###############################################################################
    # Download Information
    ###############################################################################
    def getArtistURL(self, artistRef, page=1, credit=False, unofficial=False):
        raise ValueError("Override getArtistURL")
    
    def getArtistSavename(self, discID, page=1, credit=False, unofficial=False):
        artistDir = self.disc.getArtistsDir()
        modValue  = self.dutils.getDiscIDHashMod(discID=discID, modval=self.disc.getMaxModVal())
        if modValue is not None:
            outdir    = mkSubDir(artistDir, str(modValue))
            if isinstance(page, int) and page > 1:
                outdir = mkSubDir(outdir, "extra")
                savename  = setFile(outdir, discID+"-{0}.p".format(page))
            elif credit is True:
                outdir = mkSubDir(outdir, "credit")
                savename  = setFile(outdir, discID+".p")
            elif unofficial is True:
                outdir = mkSubDir(outdir, "unofficial")
                savename  = setFile(outdir, discID+".p")
            else:
                savename  = setFile(outdir, discID+".p")
                
            return savename
        return None
    
    
    def downloadURL(self, url):
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        if self.debug:
            print("Now Downloading {0}".format(url))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need
        
        return data, response.getcode()


    def downloadArtistFromID(self, artistID, artistRef, force=False):
        if self.debug:
            print("Downloading Artist Data for ID [{0}] and Ref [{1}]".format(artistID, artistRef))
        url = self.getArtistURL(artistRef)
        savename = self.getArtistSavename(artistID)
        retval = self.downloadArtistURL(url, savename, force=force)
        

    ###############################################################################
    # Download Functions
    ###############################################################################
    def downloadArtistURL(self, url, savename, force=False, sleeptime=2):
        if isFile(savename):
            if self.debug:
                print("{0} exists.".format(savename))
            if force is False:
                return False
            else:
                print("Downloading again.")
                  
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False
            
        print("Saving {0} (force={1})".format(savename, force))
        saveFile(idata=data, ifile=savename)
        print("Done. Sleeping for {0} seconds".format(sleeptime))
        sleep(sleeptime)
        
        if isFile(savename):
            return True
        else:
            return False
    

    def downloadArtistCreditURL(self, artistData, debug=False, force=False):
        artistRef = artistData.url.url
        artistID  = artistData.ID.ID
        print("Downloading credit URL for ArtistID {0}".format(artistID))

        url      = self.getArtistURL(artistRef, credtit=True)
        savename = self.getArtistSavename(artistID, credit=True)
        if not isFile(savename) or force is True:
            retval = self.downloadArtistURL(url=url, savename=savename, force=force)
            return retval
        return False
    

    def downloadArtistUnofficialURL(self, artistData, debug=False, force=False):
        artistRef = artistData.url.url
        artistID  = artistData.ID.ID
        print("Downloading credit URL for ArtistID {0}".format(artistID))

        url      = self.getArtistURL(artistRef, unofficial=True)
        savename = self.getArtistSavename(artistID, unofficial=True)
        if not isFile(savename) or force is True:
            retval = self.downloadArtistURL(url=url, savename=savename, force=force)
            return retval
        return False
            
    

    def downloadArtistExtraURL(self, artistData, debug=False, force=False):
        newPages = 0
        pages = artistData.pages
        if pages.more is True:
            npages = pages.pages
            artistRef = artistData.url.url
            artistID  = artistData.ID.ID
            print("Downloading an additional {0} URLs for ArtistID {1}".format(npages-1, artistID))

            for p in range(2, npages+1):
                url      = self.getArtistURL(artistRef, p)
                savename = self.getArtistSavename(artistID, p)
                if not isFile(savename) or force is True:
                    self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                    newPages += 1
                    
        return newPages

            
    ################################################################################
    # Search For Artist
    ################################################################################
    def searchForArtist(self, artist):
        raise ValueError("Override searchForArtist")
        

    ################################################################################
    # Parse Artist Data
    ################################################################################
    def getArtistNumAlbums(self, artistData):
        numAlbums = sum([len(x) for x in artistData.media.media.values()])
        return numAlbums
    
    
    def parseArtistFile(ifile):
        bsdata     = getHTML(get(ifile))
        artistData = self.parse(bsdata) 
        return artistData
    
    
    
    def parseArtistModValCreditFiles(self, modVal, dbdata=None, debug=False, force=False):
        print("\t","="*100)
        print("\t","Parsing Artist Credit Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "credit")
        files  = findExt(dirVal, ext='.p')
        
        if len(files) == 0:
            return dbdata
        print("\t","  Found {0} credit files for ModVal {1}".format(len(files), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        retdbdata = False

        if dbdata is None:
            print("\t","  Loaded ", end="")
            dbdata = getFile(dbname, version=3)
            print("\t","{0} artist IDs.".format(len(dbdata)))
        else:
            retdbdata = True

        saveIt = 0
        
        nArtistMedia = {}
        print("\t","{0} artist IDs.".format(len(dbdata)))
        
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\t","\tProcessed {0}/{1} files.".format(j,len(files)))
            if debug:
                print("\t","{0}/{1} -- {2}.".format(j,len(files),ifile))
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            #currentMedia = sum([len(x) for x in dbdata[artistID].media.media.values()])
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            currentKeys = []
            if dbdata.get(artistID) is not None:
                currentKeys = list(dbdata[artistID].media.media.keys())
            else:
                dbdata[artistID] = info
                saveIt += 1
                continue
            
            keys = list(set(list(info.media.media.keys()) + currentKeys))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            if debug:
                print("\t","File:",j," \tArtist:",artistID,'-->',currentMedia,'to',sum([len(x) for x in dbdata[artistID].media.media.values()]))

                
        if retdbdata is True:
            return dbdata
        #if saveAll is False:
        #    return saveIt
                
                
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("\t","Saving {0} artists to {1}".format(len(dbdata), savename))
            print("\t","Saving {0} new (credit) artist media to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("\t","Saving {0} total (credit) artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt

    
    
    ###########################################################################################################################
    ##
    ## Extra ModVal Files
    ##
    ###########################################################################################################################
    def getArtistModValExtraFiles(self, modVal, previousDays=5, force=False):
        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "extra")
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        now    = datetime.now()
        newFiles = None
        if lastModified is None:
            newFiles = files
            print("  ===> Parsing all {0} extra files for modval {1}".format(len(newFiles), modVal))
        else:
            numFiles = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < previousDays]
            print("  ===> Found new {0} extra files (< {1} days) to parse for modval {2}".format(len(newFiles), previousDays, modVal))
        return newFiles
    
    
    
    def parseArtistModValExtraFiles(self, modVal, dbdata=None, debug=False, force=False):
        print("\t","="*100)
        print("\t","Parsing Artist Extra Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        newFiles = self.getArtistModValExtraFiles(modVal, previousDays=previousDays, force=force)
        
        if len(newFiles) == 0:
            return dbdata
        print("\t","  Found {0} extra files for ModVal {1}".format(len(newFiles), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        retdbdata = False

        if force is False:
            if dbdata is None:
                print("\t","  Loaded ", end="")
                dbdata = getFile(dbname, version=3)
                print("\t","{0} artist IDs.".format(len(dbdata)))
            else:
                retdbdata = True
        else:
            print("\t","Forcing Reloads of ModVal={0}".format(modVal))
            print("\t","  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        
        nArtistMedia = {}
        print("\t","{0} artist IDs.".format(len(dbdata)))

        
        
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 50 == 0:
                    print("\t","\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            if debug:
                print("\t","{0}/{1} -- {2}.".format(j,len(newFiles),ifile))
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            currentMedia = sum([len(x) for x in dbdata[artistID].media.media.values()])
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            keys = list(set(list(info.media.media.keys()) + list(dbdata[artistID].media.media.keys())))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            if debug:
                print("\t","File:",j," \tArtist:",artistID,'-->',currentMedia,'to',sum([len(x) for x in dbdata[artistID].media.media.values()]))

                
        if retdbdata is True:
            return dbdata
        #if saveAll is False:
        #    return saveIt
                
                
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("\t","Saving {0} new (extra) artist media to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("\t","Saving {0} total (extra) artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt

    
    
    ###########################################################################################################################
    ##
    ## Regular ModVal Files
    ##
    ###########################################################################################################################
    def getArtistModValFiles(self, modVal, previousDays=5, force=False):
        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        now    = datetime.now()
        if isFile(dbname):
            lastModified = datetime.fromtimestamp(path.getmtime(dbname))
            if force is True:
                lastModified = None
        else:
            lastModified = None

        newFiles = None
        if lastModified is None:
            newFiles = files
            print("  ===> Parsing all {0} files for modval {1}".format(len(newFiles), modVal))
        else:
            numNew    = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < previousDays]
            numRecent = [ifile for ifile in files if datetime.fromtimestamp(path.getmtime(ifile)) > lastModified]
            newFiles  = list(set(numNew).union(set(numRecent)))
            print("  ===> Found new {0} files (< {1} days) to parse for modval {2}".format(len(newFiles), previousDays, modVal))
        return newFiles
            
    
    def parseArtistModValFiles(self, modVal, previousDays=5, force=False, debug=False, doExtra=False):        
        print("-"*100)
        print("Parsing Artist Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))

        
        newFiles = self.getArtistModValFiles(modVal, previousDays=previousDays, force=force)
        if force is False:
            dbdata = getFile(dbname, version=3)
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(newFiles)))
            dbdata = {}
 
        saveIt = 0
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info

               
        forceSave = False
        if saveIt > 0 and doExtra is True:
            print("\tCalling Extra Parsing")
            dbdata = self.parseArtistModValExtraFiles(modVal, dbdata=dbdata, force=force, debug=debug)
            forceSave = True
            saveIt = len(dbdata)
            
         
        if len(self.creditToDownload) > 0:
            print("Found {0} artists that need new downloads.".format(len(self.creditToDownload)))
        

        if saveIt > 0 or forceSave is True:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("Saving {0} total artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    

    def parseArtistFiles(self, force=False, debug=False):           
        totalSaves = 0
        maxModVal  = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            saveIt = self.parseArtistModValFiles(modVal, force=force, debug=debug)
            totalSaves += saveIt
            
        print("Saved {0} new artist IDs".format(totalSaves)) 

    def parseArtistMetadataFiles(self, debug=False):   
        artistDBDir = self.disc.getArtistsDBDir()   
        maxModVal   = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            dbdata   = getFile(savename)
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
                 
        
        
    
    ################################################################################
    # Check ArtistDB Files
    ################################################################################ 
    def rmIDFiles(self, artistID):
        print("Removing files artistID {0}".format(artistID))
        savename = self.getArtistSavename(artistID)
        if isFile(savename):
            files = [savename]
        else:
            files = []
        from glob import glob
        from os.path import join
        from fileUtils import getDirname
        files += glob(join(getDirname(savename), "extra", "{0}-*.p".format(artistID)))
        print("Found {0} files to delete.".format(len(files)))
        from fsUtils import removeFile
        for ifile in files:
            removeFile(ifile)
            print("Removed File {0}".format(ifile))

                
    def rmIDsFromDBs(self, artistIDs, modValue=None):
        modvals = {}
        for artistID in artistIDs:
            modValue  = self.dutils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
            if modvals.get(modValue) is None:
                modvals[modValue] = []
            modvals[modValue].append(artistID)
            
        for modval in modvals.keys():
            dbdata = self.disc.getArtistsDBModValData(modval)
            for artistID in modvals[modval]:
                try:
                    del dbdata[artistID]
                    print("  Removed ArtistID {0}".format(artistID))
                except:
                    print("  Could not remove ArtistID {0}".format(artistID))
                    
            self.disc.saveArtistsDBModValData(modval, dbdata)
                


    def rmIDFromDB(self, artistID, modValue=None):
        print("Trying to remove data from ArtistID {0}".format(artistID))
        if modValue is None:
            modValue  = self.dutils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)
        
        saveVal = False

        if isinstance(artistID, str):
            artistID = [artistID]
        elif not isinstance(artistID, list):
            raise ValueError("Not sure what to do with {0}".format(artistID))
            
        for ID in artistID:
            try:
                del dbdata[ID]
                print("Deleted {0}".format(ID))
                saveVal = True
            except:
                print("Not there...")

            self.rmIDFiles(ID)

        if saveVal:
            print("Saving {0}".format(dbname))
            saveFile(idata=dbdata, ifile=dbname)
        else:
            print("No reason to save {0}".format(dbname))


            
    def assertDBModValUnofficialData(self, modVal, test=False):
        print("assertDBModValUnofficialData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            mediaCounts = artistData.mediaCounts.counts
            if mediaCounts.get("Unofficial") is not None:
                artistRef = artistData.url.url
                url      = self.getArtistURL(artistRef, unofficial=True)
                savename = self.getArtistSavename(artistID, unofficial=True)
                if test is True:
                    print("Download Unofficial: {0} \t".format(artistID),end="")
                if not isFile(savename):
                    if test is True:
                        print("{0} --> {1}".format(url,savename))
                        continue
                    self.downloadArtistURL(url=url, savename=savename, force=False)
                    sleep(2)
                else:
                    print("Known")


            
    def assertDBModValCreditData(self, modVal, test=False):
        print("assertDBModValCreditData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            mediaCounts = artistData.mediaCounts.counts
            if mediaCounts.get("Credits") is not None:
                artistRef = artistData.url.url
                url      = self.getArtistURL(artistRef, credit=True)
                savename = self.getArtistSavename(artistID, credit=True)
                if test is True:
                    print("Download Credit: {0} \t".format(artistID),end="")
                if not isFile(savename):
                    if test is True:
                        print("{0} --> {1}".format(url,savename))
                        continue
                    self.downloadArtistURL(url=url, savename=savename, force=False)
                    sleep(2)
                else:
                    print("Known")
                
    
    def assertDBModValExtraData(self, modVal, test=False):
        print("assertDBModValExtraData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.more is True:
                npages = pages.pages
                artistRef = artistData.url.url
                for p in range(2, npages+1):
                    url      = self.getArtistURL(artistRef, p)
                    savename = self.getArtistSavename(artistID, p)
                    if test is True:
                        print("Download Extra: {0} \t".format(artistID),end="")
                    if not isFile(savename):
                        if test is True:
                            print("{0} --> {1}".format(url,savename))
                            continue
                        self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                        sleep(2)
                    else:
                        print("Known")
                        
            
    def assertDBModValData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        if self.artistIDtoRefData is None:
            self.artistIDtoRefData = self.disc.getArtistIDToRefData()
        
        dels = []
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.redo is True and False:
                artistRef = artistData.url.url
                url       = self.getArtistURL(artistRef, 1)
                savename  = self.getArtistSavename(artistID, 1)
                self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

            ID = artistData.ID.ID
            if ID != artistID:

                nerrs += 1

                if "-" in artistID:
                    print("Extra file: {0}".format(artistID))
                    continue
                else:
                    dels.append(artistID)
                    
                    rmsavename = self.getArtistSavename(artistID)


                    ## ID = artistID                    
                    refRef      = self.artistIDtoRefData.get(artistID)
                    if refRef is None:
                        raise ValueError("Ref for ID [{0}] is None!".format(artistID))
                    else:
                        print("ArtistRef:",refRef)
                        urlRef         = self.getArtistURL(refRef)
                        savenameArtRef = self.getArtistSavename(artistID)


                    ## ID = info.ID.ID
                    try:
                        info  = self.getFileData(artistID)
                    except:
                        info  = None

                    if info is not None:
                        try:
                            refIDID      = artistIDtoRefData[info.ID.ID]
                        except:
                            refIDID      = info.url.url
                        print("ArtistID: ",refIDID)
                        urlIDID      = self.getArtistURL(refIDID)
                        savenameIDID = self.getArtistSavename(info.ID.ID)
                    else:
                        refIDID      = None
                        urlIDID      = None
                        savenameIDID = None

                        
                    if isFile(rmsavename):
                        removeFile(rmsavename)


                    if isFile(savenameArtRef):
                        removeFile(savenameArtRef)
                        self.downloadArtistURL(url=urlRef, savename=savenameArtRef, force=True, debug=True)

                    if savenameArtRef != savenameIDID:
                        if isFile(savenameIDID):
                            removeFile(savenameIDID)
                            self.downloadArtistURL(url=urlIDID, savename=savenameIDID, force=True, debug=True)


                    #print(rmsavename,'\t',savenameArtID,'\t',savenameIDID)        
        
        print("Found {0} errors with modVal {1}".format(nerrs, modVal))
        
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))
        
        for artistID in dels:
            print("Deleting {0}".format(artistID))
            try:
                del dbdata[artistID]
            except:
                continue
            
        if len(dels) > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} artist IDs to {1}".format(len(dbdata), savename))
            saveFile(idata=dbdata, ifile=savename)
        
        
    
    ################################################################################
    # Collect Metadata About Artists (4)
    ################################################################################
    def createArtistModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
    
        artistIDMetadata = {k: [v.artist.name, v.url.url] for k,v in db.items()}
        
        for artistID,artistData in db.items():
            if artistData.profile.variations is not None:
                artistIDMetadata[artistID].append([v2.name for v2 in artistData.profile.variations])
            else:
                artistIDMetadata[artistID].append([artistData.artist.name])
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-Metadata.p".format(modVal))
        
        print("Saving {0} new artist IDs name data to {1}".format(len(artistIDMetadata), savename))
        saveFile(idata=artistIDMetadata, ifile=savename)
        
        
    def createArtistAlbumModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
        
        artistIDMetadata = {}
        for artistID,artistData in db.items():
            artistIDMetadata[artistID] = {}
            for mediaName,mediaData in artistData.media.media.items():
                albumURLs  = {mediaValues.code: mediaValues.url for mediaValues in mediaData}
                albumNames = {mediaValues.code: mediaValues.album for mediaValues in mediaData}
                artistIDMetadata[artistID][mediaName] = [albumNames, albumURLs]
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-MediaMetadata.p".format(modVal))
        
        print("Saving {0} new artist IDs media data to {1}".format(len(artistIDMetadata), savename))
        saveFile(idata=artistIDMetadata, ifile=savename)

# MusixMatch

In [ ]:
#from artistMusixMatch import artistMusixMatch
from dbUtils import utilsMusixMatch
from dbBase import dbBase
import urllib
from urllib.parse import quote
from webUtils import getHTML
from fsUtils import isFile


##################################################################################################################
# Base Class
##################################################################################################################
class dbArtistsMusixMatch:
    def __init__(self, debug=False):
        self.db     = "MusixMatch"
        self.disc   = dbBase(self.db.lower())
        self.artist = artistMusixMatch(self.disc)
        self.dutils = utilsMusixMatch(self.disc)
        self.debug  = debug
        
        self.baseURL   = "https://www.musixmatch.com"        
        self.searchURL = "https://www.musixmatch.com/search/"


        
    ##################################################################################################################
    # Artist URL
    ##################################################################################################################
    def getArtistURL(self, href, page=1):
        if href.startswith("/"):
            url = "{0}{1}".format(self.baseURL, href)
        else:
            url = "{0}/{1}".format(self.baseURL, href)
        return url

        
    
    ##################################################################################################################
    #
    # Search Functions
    #
    ##################################################################################################################
    def parseSearchArtist(self, artist, data, maxArtists=99, force=False, debug=False):
        if data is None:
            return None
        
        ## Parse data
        bsdata = getHTML(data)
        
        artistDB  = {}
        
        div = bsdata.find("div", {"id": "search-results"})
        if div is not None:
            artists = div.find("div", {"class": "search-results"})
            for ul in artists.findAll("ul"):
                for li in ul.findAll("li"):
                    h2 = li.find("h2")
                    if h2 is None:
                        continue
                    ref = h2.find('a')
                    if ref is None:
                        continue
                    href = ref.attrs['href']
                    name = ref.text
                    artistID = self.dutils.getArtistID(href)
                    print("{0: <100}{1: <15}{2}".format(name,artistID,href))
                    
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": name}
                    artistDB[href]["N"] += 1
        
    
        if self.debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
            if iArtist > maxArtists:
                break
        
            discID   = self.dutils.getArtistID(href)
            url      = self.getArtistURL(href)
            savename = self.dutils.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            continue
            
            if isFile(savename):
                if force is False:
                    continue

            self.dutils.downloadArtistURL(url, savename, force=force)
            
    
    def getSearchArtistURL(self, artist):
        url = urllib.parse.urljoin(self.searchURL, quote(artist))
        url = "{0}/albums".format(url)
        return url
    
        
    def searchForArtist(self, artist, maxArtists=99, force=False, debug=False):
        print("\n\n===================== Searching For {0} =====================".format(artist))
        url = self.getSearchArtistURL(artist)
        if url is None:
            raise ValueError("URL is None!")
                    
        ## Download data
        data, response = self.dutils.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False

        self.parseSearchArtist(artist, data, maxArtists, force, debug)

In [ ]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass
from strUtils import fixName
from dbUtils import utilsMusixMatch


class artistMusixMatch(artistDBBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        self.dbUtils = utilsMusixMatch()
        
        
    ##############################################################################################################################
    ## Parse Data
    ##############################################################################################################################
    def getData(self, inputdata):
        self.getDataBase(inputdata)
        self.checkData()
        
        artist      = self.getName()
        meta        = self.getMeta()
        url         = self.getURL()
        ID          = self.getID(url)
        pages       = self.getPages()
        profile     = self.getProfile()
        media       = self.getMedia(artist.name)
        mediaCounts = self.getMediaCounts(media)
        
        err = [artist.err, meta.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc
    
    

    ##############################################################################################################################
    ## Artist Name
    ##############################################################################################################################
    def getName(self):
        for h1 in self.bsdata.findAll("h1"):
            if h1.attrs.get('title') is not None:
                artist = h1.attrs['title']
                anc = artistDBNameClass(name=artist, err=None)
                return anc

        anc = artistDBNameClass(err="NoH1")
        return anc    
    

    ##############################################################################################################################
    ## Meta Information
    ##############################################################################################################################
    def getMeta(self):
        metatitle = self.bsdata.find("meta", {"property": "og:title"})
        metaurl   = self.bsdata.find("meta", {"property": "og:url"})

        title = None
        if metatitle is not None:
            title = metatitle.attrs['content']

        url = None
        if metatitle is not None:
            url = metaurl.attrs['content']

        amc = artistDBMetaClass(title=title, url=url)
        return amc
        

    ##############################################################################################################################
    ## Artist URL
    ##############################################################################################################################
    def getURL(self):
        result1 = self.bsdata.find("link", {"rel": "canonical"})
        result2 = self.bsdata.find("link", {"hreflang": "en"})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            auc = artistDBURLClass(err=True)
            return auc

        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.MusixMatch.com", "")
            if url.find("/artist/") == -1:
                url = None
                auc = artistDBURLClass(url=url, err="NoArtist")
            else:
                auc = artistDBURLClass(url=url)
        else:
            auc = artistDBURLClass(err="NoLink")

        return auc

    

    ##############################################################################################################################
    ## Artist ID
    ##############################################################################################################################                
    def getID(self, suburl):
        discID = self.dbUtils.getArtistID(suburl.url)
        aic = artistDBIDClass(ID=discID)
        return aic


    
    ##############################################################################################################################
    ## Artist Pages
    ##############################################################################################################################
    def getPages(self):
        apc   = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        return apc
    
    

    ##############################################################################################################################
    ## Artist Variations
    ##############################################################################################################################
    def getProfile(self):          
        data = {}
        apc = artistDBProfileClass(profile=data.get("Profile"), aliases=data.get("Aliases"),
                                   members=data.get("Members"), groups=data.get("In Groups"),
                                   sites=data.get("Sites"), variations=data.get("Variations"))
        return apc

    
    
    ##############################################################################################################################
    ## Artist Media
    ##############################################################################################################################   
    def getMedia(self, artist):
        amc = artistDBMediaClass()
        name = "Albums"
        amc.media[name] = []
        
        for ul in self.bsdata.findAll("ul"):
            for li in ul.findAll("li"):
                h2 = li.find("h2")
                if h2 is None:
                    continue
                ref = h2.find('a')
                if ref is None:
                    continue
                href  = ref.attrs['href']
                album = ref.text
                #print("{0: <100}{1}".format(name,href))
                url = "https://www.musixmatch.com{0}".format(href)

                amdc = artistDBMediaDataClass(album=album, url=url, aclass=None, aformat=None, artist=artist, code=None, year=None)
                amc.media[name].append(amdc)

        return amc
    
    

    ##############################################################################################################################
    ## Artist Media Counts
    ##############################################################################################################################
    def getMediaCounts(self, media):        
        amcc = artistDBMediaCountsClass()
        
        credittype = "Releases"
        if amcc.counts.get(credittype) == None:
            amcc.counts[credittype] = {}
        for creditsubtype in media.media.keys():
            amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))
            
        return amcc

In [ ]:
ar = artistMusixMatch()

In [ ]:
from ioUtils import getFile
dbAP = dbArtistsMusixMatch()
dbAP.parseSearchArtist("Dave Matthews Band", getFile("../charts/musixmatch_searchartist.p"))

In [ ]:
retval = ar.getData("../charts/musixmatch_dmbalbums.p")
retval.show()

In [ ]:
!pip install geopandas
import geopandas

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
cities = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))
country_shapes = world[['geometry', 'iso_a3']]
country_names = world[['name', 'iso_a3']]
countries = world[['geometry', 'name']]
countries = countries.rename(columns={'name':'country'})

In [ ]:
cities

In [ ]:
countries

In [ ]:
country_names

In [ ]:
newdata = country_shapes.merge(country_names, on='iso_a3').copy(deep=True)

In [ ]:
newdata

In [ ]:
from searchUtils import findExt, findAll

In [ ]:
files = findExt("/Users/tgadfort/Downloads", ".torrent")
files

In [ ]:
opr = {'=': =, '<': <, '>': >, '>=': >=, '<='}['=']

In [ ]:
ftd = fileTimeDelta()
ftd.getDelta("< 2 Hours")
ftd.show()

In [ ]:
from searchUtils import filesFromDir
from recentFilesUtils import recentFiles

In [ ]:
ffd   = filesFromDir(ext=".p")
files = ffd.getFiles("/Users/tgadfort/discogs/artists-rateyourmusic/9")

In [ ]:
rf = recentFiles()
rf.setFiles(files)
retval = rf.getFilesByModTime('< 0 Days', '/Users/tgadfort/dbdiscogs/artists-rateyourmusic-db/9-DB.p')
retval

In [ ]:
rf = recentFiles()
rf.setFiles(files)
retval = rf.getFilesByRecency('> 1 Days')
retval

In [ ]:
#for ifile, fmtd in tmp.items():
#    print("{0}\t{1}  \t{2}\t\t{3}".format(ifile, fmtd.getDeltaNow(), ftd.td, ftd.isTrue(fmtd.getDeltaNow())))

In [ ]:


rf = recentFiles()
rf.setFiles(files)
rf.findFilesByRecentTime(minutes=60)
#rf.

In [ ]:
lastModified

In [ ]:
ifile="/Users/tgadfort/Downloads/Strauss,.Johann.II.(1825-1899).[Austria].torrent"
from os.path import getmtime

In [ ]:
lastModified = datetime.fromtimestamp(getmtime(ifile))
lastModified

# Metal Archives

In [ ]:
import urllib
def downloadURL(url):

    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    return data, response.getcode()


In [ ]:
from webUtils import getURL

searchURL = "https://www.metal-archives.com/search?searchString=Metalica&type=band_name"
headers = {'User-Agent':"Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"}
req = urllib.request.Request(searchURL, headers=headers)
response = urllib.request.urlopen(req)
#data, code = downloadURL(searchURL)

In [ ]:
response.read()

# Classical Archives